In [1]:
import numpy as np
import simpy
from scipy.stats import expon, gamma, uniform

In [2]:
def simulate(lam_dt, lam_st, lam_eat, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        rqt_rest = restaurant.request()
        yield rqt_rest
        t0 = env.now
        kitchen_process = env.process(kitchen_pr())
        bar_process = env.process(bar_pr())
        yield simpy.AllOf(env, [kitchen_process, bar_process])
        dt_eat = expon.rvs(scale=1/lam_eat)
        yield env.timeout(dt_eat)
        restaurant.release(rqt_rest)
        rev = gamma.rvs(5, scale=1/30) # spend money with gamma distribution
        X.append(rev)

    def kitchen_pr():
        rqt = kitchen.request()
        yield rqt
        yield env.timeout(expon.rvs(scale=1/lam_st))
        kitchen.release(rqt)

    def bar_pr():
        rqt = bar.request()
        yield rqt
        yield env.timeout(expon.rvs(scale=1/lam_st))
        bar.release(rqt)
    


    X = []
    env = simpy.Environment()
    restaurant = simpy.Resource(capacity=10, env=env)
    kitchen = simpy.Resource(capacity=1, env=env)
    bar = simpy.Resource(capacity=1, env=env)
    env.process(arrivals())
    env.run()
    return np.sum(X)


In [3]:
# 10 tables in restaurant
# party places orders at bar and kitchen
# when they get both orders the occupy the table for an exponential amount of time
# they spend a gamma amount of money
# compute the revenue of the restaurant for each simulation

simulate(50, 10, 10, 1)


8.289158059531102

In [4]:
def simulate(lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        if uniform.rvs() < 0.2: # 20% request are high priority
            priority = 0 # high priority
        else:
            priority = 1 # regular
        rqt = server.request(priority=priority) # cut the line but not kick off anyone at service
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.PriorityResource(capacity=1, env=env) # priority resource
    env.process(arrivals())
    env.run()
    return np.max(X)





In [5]:
def simulate(lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        if uniform.rvs() < 0.2:
            priority = 0
            preempt = True
        else:
            priority = 1
            preempt = False
        rqt = server.request(priority=priority, preempt=preempt)
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.PreemptiveResource(capacity=1, env=env) # preemptive resource: does kick off people in the resource
    env.process(arrivals())
    env.run()
    return np.max(X)





In [6]:
## in service: priority 1, preempt=False
## new request: priority 0, preempt=True
## -> kick off



## in service: priority 1, preempt=False
## in line: priority 0, preempt=False
## new request: priority 0, preempt=True # blocked
## -> don't cut in line, so also don't kick off 












